In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Introduction

This notebook is a short, practical introduction to creating AI agents using the `google.adk` package. 

It shows how to configure credentials, build a simple agent, run it with an in-memory runner, and optionally launch the ADK web UI for interactive debugging.

Derived from Day 1 of Google's "5-day AI Agents Intensive" codelab, this version is adapted so you can run it outside the original environment (e.g., locally or on Kaggle). Requirements: Python, the `google-adk` package, and a valid `GOOGLE_API_KEY` (the notebook will prompt or read from the environment if needed).

Sections covered:
- Setup & authentication
- ADK imports and helper utilities
- Define an agent and runner
- Run/debug the agent and launch the ADK web UI

See the ADK docs for more details: https://google.github.io/adk-docs/

## ⚙️ Section 1: Setup

### Install dependencies and Configure Gemini API Key
```

In [ ]:
# install the Google ADK library
%pip install google-adk --quiet

### 1.1 Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to as environment variable

`GOOGLE_API_KEY`

In [ ]:
import os
import dotenv

# Load environment variables from .env file if it exists
dotenv.load_dotenv()

# read from environment variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# if missing, prompt user to enter the API key
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Enter your Google API key: ")

if not GOOGLE_API_KEY:
    print("❌ Error: Google API key is required. Please set the 'GOOGLE_API_KEY' environment variable or enter it when prompted.")
else:
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Google API key obtained successfully.")


### 1.2 Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

### 1.3 Helper functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

---

## 🤖 Section 2: Your first AI Agent with ADK

### 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

### 2.2 Define your agent

Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [ ]:
research_agent = Agent(
    name="ResearchAssistant",
    model="gemini-2.5-flash-lite",
    description="Fetches and summarizes latest info on a given topic.",
    instruction="You are a research assistant. Given a particular topic, use Google Search for current info and summarize the search results into a concise 4-bullet brief.",
    tools=[google_search],
)

print("✅ Research Agent defined.")

### 2.3 Run your agent

Now it's time to bring your agent to life and send it a query. To do this, you need a [`Runner`](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [ ]:
runner = InMemoryRunner(agent=research_agent)

print("✅ Runner created.")

👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the documentation related to [runtime in ADK](https://google.github.io/adk-docs/runtime/).

**b. Now you can call the `.run_debug()` method to send our prompt and get an answer.**

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [ ]:
response = await runner.run_debug("quantum computing trends 2025")

You can see a summary of ADK and its available languages in the response.

### 2.4 How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

1. The agent inspects and is aware of which tools it has available to use.
2. The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the **ADK web UI**, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

---

## 💻 Section 3: Try the ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

In [ ]:
!adk create research_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

Now we can run ADK web:

In [ ]:
!adk web

---

## ✅ Congratulations!

You've built and run your first agent with ADK! You've just seen the core concept of agent development in action.


### 📚 Learn More

Refer to the following documentation to learn more:

- [ADK Documentation](https://google.github.io/adk-docs/)
- [ADK Quickstart for Python](https://google.github.io/adk-docs/get-started/python/)
- [ADK Agents Overview](https://google.github.io/adk-docs/agents/)
- [ADK Tools Overview](https://google.github.io/adk-docs/tools/)